In [13]:
import pandas as pd

from solution import *

/home/engineer/anaconda/envs/palamariuk-genai/lib/python3.11/site-packages/pandas/util/_decorators.py:380: RuntimeWarning: coroutine 'Client.aquery' was never awaited
  component.format(**params)


In [8]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
# Load dataset
def load_dataset(filepath):
    with open(filepath, "r", encoding="utf-8") as file:
        return [json.loads(line) for line in file]

In [5]:
dataset = load_dataset("dataset.jsonl")

In [14]:
pd.DataFrame(dataset)

,input,output,type,expected_answer
0,Дробові рівняння,Розв'яжіть рівняння: 1/(x+2) + 1/(x-2) = 1/3,задача,x = 4 або x = -4
1,Теорема Піфагора,"Знайдіть гіпотенузу трикутника, якщо його кате...",задача,10 см
2,Формули скороченого множення,Розкладіть на множники вираз: x^2 - 9.,приклад,(x - 3)(x + 3)
3,Лінійні рівняння,Розв'яжіть рівняння: 5x - 7 = 13.,задача,x = 4
4,Пропорції,"Знайдіть x, якщо 4 : 6 = x : 9.",задача,x = 6
...,...,...,...,...
99,Формули скороченого множення,Розкладіть на множники: x^2 - 6x + 9.,приклад,(x - 3)²
100,Системи рівнянь,"Розв'яжіть систему рівнянь: x + y = 5, x - y = 1.",задача,"x = 3, y = 2"
101,Статистика,"Знайдіть медіану набору чисел: 2, 3, 5, 8, 11.",задача,5
102,Логарифми,Обчисліть log₅25.,задача,2


In [10]:
results = []

for entry in tqdm.tqdm(dataset, desc="Evaluating tasks"):
    input_task = entry["output"]
    expected_answer = entry["expected_answer"]

    # Format task for WolframAlpha
    formatted_task = format_task_for_wolfram(input_task)

    try:
        # Solve the task
        obtained_answer = solve_task(formatted_task)
    except:
        obtained_answer = None

    # Log results
    results.append({
        "input": entry["input"],
        "output": input_task,
        "expected_answer": expected_answer,
        "obtained_answer": obtained_answer
    })

Evaluating tasks:   0%|          | 0/104 [00:00<?, ?it/s]2025-01-10 15:16:33,686 - INFO - Reformatting task for WolframAlpha using LLM.
2025-01-10 15:16:35,129 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 15:16:35,137 - INFO - Attempting to solve the task with WolframAlpha.
2025-01-10 15:16:37,038 - INFO - HTTP Request: GET https://api.wolframalpha.com/v2/query?appid=H6KXWQ-5VTJPYJGPQ&input=solve%281%2F%28x%2B2%29%20%2B%201%2F%28x-2%29%20%3D%201%2F3%2C%20x%29 "HTTP/1.1 200 OK"
Evaluating tasks:   1%|          | 1/104 [00:03<05:45,  3.35s/it]2025-01-10 15:16:37,042 - INFO - Reformatting task for WolframAlpha using LLM.
2025-01-10 15:16:38,557 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 15:16:38,567 - INFO - Attempting to solve the task with WolframAlpha.
2025-01-10 15:16:41,796 - INFO - HTTP Request: GET https://api.wolframalpha.com/v2/query?appid=H6KXWQ-5VTJPYJGPQ&input=hypote

In [11]:
results

[{'input': 'Дробові рівняння',
  'output': "Розв'яжіть рівняння: 1/(x+2) + 1/(x-2) = 1/3",
  'expected_answer': 'x = 4 або x = -4',
  'obtained_answer': 'Input interpretation - solve 1/(x + 2) + 1/(x - 2) = 1/3 for x\nResults - x = 3 - sqrt(13)\nPlot - None\nNumber line - None'},
 {'input': 'Теорема Піфагора',
  'output': 'Знайдіть гіпотенузу трикутника, якщо його катети дорівнюють 6 см і 8 см.',
  'expected_answer': '10 см',
  'obtained_answer': 'Input interpretation - rectangle | width 6 cm (centimeters)\nheight 8 cm (centimeters) | hypotenuse length\nResult - (data not available)\nVisual representation - None\nDefining inequalities - -3<=x - x_0<=3 and -4<=y - y_0<=4\n(assuming center (x_0, y_0))\nCombinatorial properties - vertices | 4\nedges | 4\nProperties - independent edge lengths | 6 cm (centimeters) | 8 cm (centimeters)\ndiagonal length | 10 cm (centimeters)\narea | 48 cm^2 (square centimeters)\nperimeter | 28 cm (centimeters)\ninterior angle | 90° = π/2 rad≈1.571 rad\ninteri

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize LLM
llm = ChatOpenAI(temperature=0.0, model="gpt-4")

# Define comparison prompt
comparison_prompt = PromptTemplate(
    input_variables=["expected_answer", "obtained_answer"],
    template=(
        "Compare the expected answer and the obtained answer from WolframAlpha.\n\n"
        "Expected Answer: {expected_answer}\n"
        "Obtained Answer: {obtained_answer}\n\n"
        "Determine if they match. Return 'True' if they match exactly or semantically, "
        "otherwise return 'False'. Only output 'True' or 'False'."
    ),
)

# LLM Chain for comparison
comparison_chain = LLMChain(llm=llm, prompt=comparison_prompt)

# Add 'correct' field
for result in results:
    comparison = comparison_chain.run({
        "expected_answer": result["expected_answer"],
        "obtained_answer": result["obtained_answer"]
    })
    result["correct"] = comparison.strip() == "True"

2025-01-10 16:33:36,168 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:36,891 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:37,669 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:38,611 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:39,390 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:40,019 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:40,984 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:41,871 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:33:42,964 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [31]:
results_df = pd.DataFrame(results)
results_df.head(30)

,input,output,expected_answer,obtained_answer,correct
0,Дробові рівняння,Розв'яжіть рівняння: 1/(x+2) + 1/(x-2) = 1/3,x = 4 або x = -4,Input interpretation - solve 1/(x + 2) + 1/(x ...,False
1,Теорема Піфагора,"Знайдіть гіпотенузу трикутника, якщо його кате...",10 см,Input interpretation - rectangle | width 6 cm ...,True
2,Формули скороченого множення,Розкладіть на множники вираз: x^2 - 9.,(x - 3)(x + 3),Input interpretation - factor | x^2 - 9\nResul...,True
3,Лінійні рівняння,Розв'яжіть рівняння: 5x - 7 = 13.,x = 4,Input interpretation - solve 5 x - 7 = 13\nRes...,True
4,Пропорції,"Знайдіть x, якщо 4 : 6 = x : 9.",x = 6,Input interpretation - solve 4/6 = x/9 for x\n...,True
5,Відсотки,Скільки відсотків становить 25 від числа 200?,12.5%,Input interpretation - 25×1%×200\nResult - 50,False
6,Квадратні рівняння,Розв'яжіть рівняння: x^2 - 5x + 6 = 0.,x = 2 або x = 3,Input interpretation - solve x^2 - 5 x + 6 = 0...,True
7,Графіки функцій,Накресліть графік функції y = 2x + 3.,"Пряма, що проходить через точки (0, 3) і (1, 5)",Input interpretation - plot | y = 2 x + 3\nPlo...,True
8,Об'єм тіла,Знайдіть об'єм циліндра з радіусом основи 5 см...,785.4 см^3 (π ≈ 3.14),Input interpretation - cylinder | base radius ...,True
9,Системи рівнянь,"Розв'яжіть систему рівнянь: x + y = 10, x - y ...","x = 7, y = 3",Input interpretation - solve x + y = 10\nx - y...,True


In [28]:
results_df[results_df['correct']].shape[0] / results_df.shape[0]

0.7980769230769231